# Data Preprocessing



In [176]:
import re
import json
# validate the dict data for hero and item
def validate(data):
    id=[]
    for key in data:
        tid=data[key]
        id.append(tid)
    id.sort();
    flag=True
    for i in range(0, len(id)):
        if id[i]!=i+1:
            print i
            flag=False
    #print id
    return flag
# get the key-id pairs of hero or item data dict to be continuous
def continuous_format(data):
    i=0
    for key in data:
        data[key]=i
        i+=1

# construct item/hero-id based on json data
# reformat the id sequence if iflag is True
# iflag=True=>1. id reformat, 2. upgrade item check, 3. repeat name ignore
# iflag=False=>1. repeat name ignore
def dict_from_json(data, iflag=False):
    id_data=dict()
    cost_data=dict()
    repeat=0
    for i in range(0, len(data)):
        # substitute prefix part
        name=re.sub('(npc_dota_hero_|item_)', '', data[i]['name'])
        iid=data[i]['id']
        # not repeated item/hero
        if name not in id_data:
            if iflag:
                #if (is_upgrade_item(iid)==-1):
                cost_data[name]=int(data[i]['cost'])
                id_data[name]=i-repeat
                #else:
                #    print "upgraded item: " + str(name) + "\t" + str(iid)
                #    repeat+=1
            else:
                id_data[name]=int(data[i]['id'])
        else:
            print "repeated item: " + str(name) + "\t" + str(iid)
            repeat+=1
    return id_data, cost_data

def is_upgrade_item(iid):
    up_lst = [201,202,203,204,193,194,220]
    up_dic = {201:104,202:104,203:104,204:104,193:106,194:106,220:48}
    if iid in up_lst:
        return up_dic[iid]
    else:
        return -1

# for a up-to-date item/hero list, we can http request to
# https://api.opendota.com/api/explorer?sql=SELECT%20*%20FROM%20items
heroes=json.load(open('heroes.json'))['rows']
items=json.load(open('items.json'))['rows']

# assumes: v in <k, v> is a int number
# reverse dict with <k, v> pairs to
# new array where v as index
# assumes: value is not continuous
import operator
def reverse_dict(data):
    mkey=max(data.iteritems(), key=operator.itemgetter(1))[0]
    mval=int(data[mkey])
    res=['']*(mval+1)
    for key in data:
        res[data[key]]=key
    return res

heroes, emp=dict_from_json(heroes, False)
items, icosts=dict_from_json(items, True)
iid2name=reverse_dict(items)
hid2name=reverse_dict(heroes)
        

#if validate(heroes):
#    print "heroes continuous id assignment success!"
#if validate(items):
#    print "items continuous id assignment success!"
#print items
#print heroes
#print items

repeated item: spirit_vessel	267
repeated item: recipe_spirit_vessel	195


In [175]:
print items['demon_edge']
print icosts['demon_edge']
print heroes['antimage']

155
2200
1


In [205]:
# based on the google drive data
# hero count=113
# item count=1020
import json
import os
import sys

fpath='./data/'

# init some coefficients
# the baseline freq coefficient
a=1.2
b=0.8

# init basic hero-item freq matrix

def maxValInDict(data):
    maxval=(-sys.maxint-1)
    for key in data:
        if data[key]>maxval:
            maxval=data[key]
    return maxval

def count_purchased_items(purchases, price_threshold=500):
    total_purchase_num = 0
    for item_name, num_purchased in purchases.items():
        if num_purchased is None:
            continue
            
        item_cost = icosts[item_name]
        if item_cost >= price_threshold:
            total_purchase_num += num_purchased
    return total_purchase_num

hcount=maxValInDict(heroes)+1
icount=len(items)

hero_num_item_records = []

print "hero count: " + str(hcount)
print "item count: " + str(icount)
basic_freq=[]
for i in range(0, hcount):
    basic_freq.append([0]*icount)
    hero_num_item_records.append([])
    
num_records = len(os.listdir(fpath))

# load from data and computer base freq
# the first five heroes in the players list is radiant
for fname in os.listdir(fpath):
    data=json.load(open(fpath+fname))
    players=data['players']
    for i in range(0, len(players)):
        player=players[i]
        heroid=player['hero_id']
        purchase=player['purchase']
        
        num_purchased_items = count_purchased_items(purchase)
        hero_num_item_records[heroid].append(num_purchased_items)
        
        if player['isRadiant']:
            win=player['radiant_win']
        else:
            win=(not player['radiant_win'])
        # be noticed that purchase is a dictionary
        for key in purchase:
            iid=items[key]
            # two conditions
            # 1. if vital, temporarily cost>=500
            # 2. if not null
            if (icosts[key]>=500) and (purchase[key]!=None):
                try:
                    if win: 
                        basic_freq[heroid][iid]+=a
                    else: 
                        basic_freq[heroid][iid]+=b
                except IndexError as e:
                    print "file name: " + fname
                    print "item id: " + str(iid)
                    print "hero id: " + str(heroid)

hero_item_count = []
for item_count_list in hero_num_item_records:
    if len(item_count_list) == 0:
        hero_item_count.append(0)
    else:
        hero_item_count.append(int(round(sum(item_count_list) / float(len(item_count_list)))))
print hero_item_count

hero count: 121
item count: 292
[0, 22, 16, 13, 20, 10, 18, 10, 21, 18, 19, 22, 20, 16, 12, 17, 12, 24, 22, 16, 15, 16, 19, 17, 0, 18, 9, 12, 12, 17, 12, 12, 15, 14, 20, 21, 14, 10, 12, 17, 14, 21, 19, 14, 19, 15, 24, 18, 24, 18, 13, 15, 12, 19, 20, 14, 15, 19, 17, 18, 12, 11, 14, 19, 10, 17, 13, 20, 11, 14, 15, 17, 20, 18, 19, 14, 13, 15, 13, 11, 12, 17, 11, 12, 11, 15, 9, 8, 16, 12, 11, 11, 15, 17, 21, 20, 12, 11, 21, 20, 10, 14, 18, 12, 16, 19, 18, 8, 17, 20, 13, 8, 13, 13, 18, 0, 0, 0, 0, 13, 19]


In [212]:
# predict based on model
# the basic model is a matrix that contains weight 
# returns base h*k item recommendations
import random
from scipy import spatial as sp
import numpy as np
from __future__ import division
# if we do not consider time, rather we consider 
# all the necessary items in all time, then our method should also be effective
# in this case we may recommend more items like 15-20 that captures early-mid-final 
# k can be determined following:
# 1. get the avg total items throughout every match for every hero
# 2. guess
def base_rec(heroes, model, k):
    res=[]
    for h in heroes:
        hifreq=model[h]
        hifreq=hifreq.sort()
        r1=hifreq[0:k]
        r1=dict((r, 1) for r in r1)
        res.append(r1)
    return res

def base_rec_h(h, model, k):
    hifreq=model[h]
    #print "hifreq: "
    #print hifreq
    #print "k is: " + str(k)
    #print "top k index: "
    
    tki=topk_index(hifreq, k)
    print "recommended length: " + str(len(tki))
    return tki

def topk_index(arr, k):
    arr=np.array(arr)
    return arr.argsort()[-k:][::-1]

# in matches won, recommend items, compute similarity with actual items
# OR
# in all games, find which side had items more similar to recommended, then compute winning percentage
# calculate the similarity between item purchase log
# per hero

# sim: similarity between item purchases
# ideal evaluation: P(Exactly same purchase log) 
# @hp: actual hero-purchase counter dict array
# @hp_rec: recommended hero-purchase counter dict array
# @opt: aggregation function, average and etc
# TODO: for diff heroes, we may have different weight when calc total similarity
def team_purchase_sim_calc(hp, hp_rec, norm=False, aggr_opt='avg'):
    #print "len(hero purchase): " + str(len(hp))
    sim_vec=[]
    tot_sim=0
    for (h, hpr) in zip(hp, hp_rec):
        #print h
        #print hpr
        # item purchase counter to feature vector
        h, hpr=feature_vec(h, hpr)
        # do normalization if needed
        if norm:
            norm1=np.linalg.norm(hp)
            norm2=np.linalg.norm(hpr)
            h=h/norm1
            hpr=hpr/norm2
        # calc cosine similarity
        sim=sp.distance.cosine(h, hpr)
        # append current hero item similarity
        sim_vec.append(sim)
    print "per hero similarity vector:"
    print sim_vec
    if aggr_opt=='avg':
        #print len(hp)
        # the length should always be 5
        assert(len(hp)==5)
        #print "sim_vec: "
        #print sim_vec
        tot_sim=sum(sim_vec)/len(sim_vec)
    else:
        print "no such aggr function is pre defined!"
        tot_sim=-1
    return tot_sim

# assumes: we have tot_count[h] that stores the avg total "vital" item purchased by hero h
# assumes: dummy_is_vital(iid)
# necissity evaluation
# calculating probability 
def nec_eva(fpath, model, items, tot_count, pred=base_rec_h):
    mcount=0
    sim_sum=0
    for fname in os.listdir(fpath):
        data=json.load(open(fpath+fname))
        wplayers=[]
        # assumes: the first 5 is radiant hero
        # get the winner players
        if(data['radiant_win']):
            wplayers=data['players'][0:5]
        else:
            wplayers=data['players'][5:10]
        hero_vitem=[]
        rec_vitem=[]
        for p in wplayers:
            # vital items that we consider
            vitem=dict()
            purchase=p['purchase']
            hid=p['hero_id']
            #print "purchase length of hero " + str(hid) + ": " + str(len(purchase))
            for k in purchase:
                iid=items[k]
                if icosts[k]>=500:
                #if dummy_is_vital(iid):
                    vitem[k]=purchase[k]
            hero_vitem.append(vitem)
            print hid2name[hid]
            print "actual purchase: "
            print vitem
            #print "hero item avg count: " + str(hero_item_count[hid])
            rec=base_rec_h(hid, model, hero_item_count[hid])
            rec_vitem.append(rec)
            # print recommended items
            print "recommended: "
            rec_name=[iid2name[iid] for iid in rec]
            print rec_name
            print ""
        sim=team_purchase_sim_calc(hero_vitem, rec_vitem)
        print "rec-actual item purchase similarity of match " + str(fname) + ": " + str(sim)
        if not np.isnan(sim):
            sim_sum=(sim_sum*mcount+sim)/(mcount+1)
            mcount+=1
    print "all winners similarity avg: " + str(sim_sum)
# sufficiency evaluation
#def suf_eva():
# utils
# transform two dict to feature vector
# dic1: <iname, 1/0>
# arr2: list of iname recommended
def feature_vec(dic1, arr2):
    kvec=[]
    vec1=[]
    for k, v in dic1.iteritems():
        kvec.append(k)
        # changed from counter to 1
        vec1.append(1)
    vec2=[0]*len(kvec)
    for iid in arr2:
        name=iid2name[iid]
        if iid2name[iid] in kvec:
            index=kvec.index(name)
            vec2[index]=1
        else:
            kvec.append(name)
            vec1.append(0)
            vec2.append(1)
    return vec1, vec2
# backup
#def dict_to_vec(dic1, dic2):
#    kvec=[]
#    vec1=[]
#    for k, v in dic1.iteritems():
#        kvec.append(k)
#        # changed from counter to 1
#        vec1.append(1)
#    vec2=[0]*len(kvec)
#    for k, v in dic2.iteritems():
#        if k in kvec:
#            index=kvec.index(k)
#            if v==None:
#                v=0
#            vec2[index]=1
#        else:
#            kvec.append(k)
#            vec1.append(0)
#            vec2.append(1)
#    return vec1, vec2
nec_eva('test/', basic_freq, items, hero_item_count)
    

pudge
actual purchase: 
{u'tranquil_boots': 1, u'soul_ring': 1, u'boots': 1, u'blink': 1}
recommended length: 12
recommended: 
[u'boots', u'tranquil_boots', u'blink', u'ring_of_health', u'void_stone', u'energy_booster', u'recipe_aether_lens', u'aether_lens', u'staff_of_wizardry', u'broadsword', u'cloak', u'chainmail']

vengefulspirit
actual purchase: 
{u'dragon_lance': 1, u'invis_sword': 1, u'claymore': 1, u'boots': 1, u'chainmail': 1, u'medallion_of_courage': 1, u'shadow_amulet': 1, u'talisman_of_evasion': 1, u'ogre_axe': 1, u'solar_crest': 1}
recommended length: 15
recommended: 
[u'boots', u'staff_of_wizardry', u'chainmail', u'medallion_of_courage', u'ring_of_health', u'force_staff', u'ogre_axe', u'power_treads', u'dragon_lance', u'gloves', u'talisman_of_evasion', u'energy_booster', u'hurricane_pike', u'recipe_aether_lens', u'void_stone']

nevermore
actual purchase: 
{u'power_treads': 1, u'quarterstaff': 1, u'boots': 1, u'recipe_black_king_bar': 1, u'shadow_amulet': 1, u'invis_sword'

In [140]:
print basic_freq[1]

[6.8, 0, 0, 0, 0, 21.200000000000006, 27.600000000000005, 2.4000000000000004, 0.8, 0, 0.8, 27.600000000000005, 0, 1.2, 0.8, 4.0, 4.0, 2.4, 26.000000000000007, 0, 15.199999999999998, 0, 0, 27.600000000000005, 0, 0, 0, 26.000000000000007, 0, 6.0, 0, 0, 1.2, 0, 1.2, 0, 0, 0, 3.2, 0, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.8, 0, 0, 0, 3.2, 0, 1.2, 0, 25.200000000000006, 1.6, 0, 0, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.2, 3.2, 0, 0, 15.2, 26.000000000000007, 19.6, 0.8, 3.2, 26.000000000000007, 3.2, 27.600000000000005, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 15.2, 0, 20.400000000000002, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.8, 0, 0, 0, 0, 0, 0, 0, 2.0, 4.8, 0, 0, 6.8, 0, 0, 0, 0, 0, 1.2, 0, 0, 0, 0.8, 25.200000000000006, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.2, 8.0, 0, 26.000000000000007, 3.2, 0, 0, 0.8, 0, 0, 0, 0, 1.2, 0, 0, 0, 13.599999999999998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.8, 0, 0, 26.000000000000007, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13.59999999999

In [141]:
# testing cosine similarity calculation
from scipy import spatial as sp
import numpy as np
data=json.load(open('data/3803000030.json'))
dict1=data['players'][0]['purchase']
dict2=data['players'][1]['purchase']

vec1, vec2=dict_to_vec(dict1, dict2)

print "unnormalized: "
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim

print "normalized: "
norm1=np.linalg.norm(vec1)
norm2=np.linalg.norm(vec2)
vec1=vec1/norm1
vec2=vec2/norm2
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim


unnormalized: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.582634993816
normalized: 
[ 0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.1796053  0.1796053  0.         0.         0.1796053  0.         0.
  0.1796053  0.         0.1796053  0.1796053  0.         0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053 

In [117]:
# testing cosine similarity calculation
from scipy import spatial as sp
import numpy as np
data=json.load(open('data/3803000030.json'))
dict1=data['players'][0]['purchase']
dict2=data['players'][1]['purchase']

vec1, vec2=dict_to_vec(dict1, dict2)

print "unnormalized: "
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim

print "normalized: "
norm1=np.linalg.norm(vec1)
norm2=np.linalg.norm(vec2)
vec1=vec1/norm1
vec2=vec2/norm2
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim


unnormalized: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.582634993816
normalized: 
[ 0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.1796053  0.1796053  0.         0.         0.1796053  0.         0.
  0.1796053  0.         0.1796053  0.1796053  0.         0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053 

In [118]:
# evaluate difference between purchase_log and purchase
data=json.load(open('data/3803000030.json'))
exist=False
for p in data['players']:
    kdict=dict()
    for i in p['purchase_log']:
        key=i['key']
        if key in kdict:
            kdict[key]+=1
        else:
            kdict[key]=1
    print "purchase: " + str(len(p['purchase'])) + "\tlog: " + str(len(kdict))
    print "keys not in purchase log: "
    for key in p['purchase']:
        if key not in kdict:
            print key
    print ""
    #print "keys not in purchase: "
    #for key in kdict:
    #    if key not in p['purchase']:
    #        print key
    #        exist=True
#print exist

# TODO: evaluate if 0-4 is radiant, 5-9 is dire

purchase: 15	log: 15
keys not in purchase log: 

purchase: 31	log: 28
keys not in purchase log: 
dust
recipe_cyclone
recipe_hand_of_midas

purchase: 23	log: 21
keys not in purchase log: 
dust
recipe_yasha

purchase: 22	log: 18
keys not in purchase log: 
recipe_maelstrom
recipe_hand_of_midas
dust
recipe_diffusal_blade

purchase: 25	log: 22
keys not in purchase log: 
dust
recipe_diffusal_blade
recipe_yasha

purchase: 14	log: 12
keys not in purchase log: 
recipe_soul_ring
dust

purchase: 21	log: 21
keys not in purchase log: 

purchase: 37	log: 33
keys not in purchase log: 
dust
recipe_black_king_bar
recipe_sange
recipe_yasha

purchase: 38	log: 33
keys not in purchase log: 
recipe_manta
dust
recipe_wraith_band
recipe_yasha
recipe_force_staff

purchase: 19	log: 17
keys not in purchase log: 
recipe_force_staff
ward_dispenser



In [119]:
lst=[1, 2, 3, 54]
print lst[0:3]

[1, 2, 3]
